# Introduction


**What?** Training on augmented data



# Theoretical recall - data augmentation


- The best way to improve the performance of a machine learning model is to train it on more data. 
- The more examples the model has to learn from, the better it will be able to recognize which differences in images matter and which do not. 
- More data helps the model to *generalize* better.
- One easy way of getting more data is to use the data you already have. 
- If we can transform the images in our dataset in ways that preserve the class, we can teach our classifier to ignore those kinds of transformations. 
- For instance, whether a car is facing left or right in a photo doesn't change the fact that it is a *Car* and not a *Truck*. 
- So, if we **augment** our training data with flipped images, our classifier will learn that "left or right" is a difference it should ignore.
- And that's the whole idea behind data augmentation: add in some extra fake data that looks reasonably like the real data and your classifier will improve.



# Import modules

In [ ]:
import pandas as pd
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import VGG16

In [ ]:
# Reproducability
def set_seed(seed = 31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore")

# Load the dataset

In [ ]:
# Load training and validation sets
ds_train_ = image_dataset_from_directory(
    '../DATASETS/car-or-truck/train',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)
ds_valid_ = image_dataset_from_directory(
    '../DATASETS/car-or-truck/valid',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

# Data Pipeline


def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label


AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

# Building the model with data augmentation


- Keras lets you augment your data in two ways. 
- The first way is to include it in the data pipeline with a function like ImageDataGenerator. The second way is to include it in the model definition by using Keras's preprocessing layers. 
- This is the approach that we'll take.



In [ ]:
# , input_shape=[(128, 128, 1)])
pretrained_base = VGG16(include_top=False, weights="imagenet")

pretrained_base.trainable = False

model = keras.Sequential([
    # Preprocessing
    preprocessing.RandomFlip('horizontal'),  # flip left-to-right
    preprocessing.RandomContrast(0.5),  # contrast change by up to 50%
    # Base
    pretrained_base,
    # Head
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

# Train and Evaluate

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=30,
    verbose=1,
)

In [ ]:
history_frame = pd.DataFrame(history.history)

history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()


- The training and validation curves in the model from Tutorial 1 diverged fairly quickly, suggesting that it could benefit from some regularization. 
- The learning curves for this model were able to stay closer together, and we achieved some modest improvement in validation loss and accuracy. 
- This suggests that the dataset did indeed benefit from the augmentation.



# References


- https://www.kaggle.com/ryanholbrook/data-augmentation
- [Link to dataset](https://www.kaggle.com/ryanholbrook/car-or-truck)
- [How to import the pre-trained model]()

